In [1]:
import sys
sys.path.append('/source/main')

In [2]:
import logging
logging.basicConfig(level=logging.INFO)
import time
from itertools import chain

import torch
from torch import nn
import numpy as np
import pandas as pd
pd.set_option('display.max_colwidth', -1)

from preprocess import preprocessor
from model_def.seq2seq_beamsearch_attn_with_src import Seq2SeqBeamAttnWithSrc
from utils import pytorch_utils, text_utils
from data_for_train import dataset as my_dataset
from train import trainer

# Setup and load model

In [3]:
my_dataset.bootstrap()

INFO:root:Indexing vocabs successfully. Total vocabs: 25390
INFO:root:Indexing vocabs successfully. Total vocabs: 50437
INFO:root:Vocab for source from file /source/main/vocab/output/src.pkl contains 25390 tokens
INFO:root:Vocab for source from file /source/main/vocab/output/tgt.pkl contains 50437 tokens


In [5]:
my_dataset.voc_tgt.index2word[0]

'__p__'

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = Seq2SeqBeamAttnWithSrc(src_vocab_size=len(my_dataset.voc_src.index2word),
                    tgt_vocab_size=len(my_dataset.voc_tgt.index2word),
                    start_idx=2,
                    end_idx=3, beam_width=1, device=device
                    )
model.to(device)
PRE_TRAINED_MODEL='/source/main/train/output/saved_models/Seq2SeqAttnWithSrc/2019-06-01T16:23:45/470000.pt'
checkpoint = torch.load(PRE_TRAINED_MODEL, map_location=device)
model.load_state_dict(checkpoint['model_state_dict'], strict=False)
logging.info('Load pre-trained model from %s successfully', PRE_TRAINED_MODEL)

model.eval()
logging.info('Model architecture: \n%s', model)
logging.info('Total trainable parameters: %s', pytorch_utils.count_parameters(model))

INFO:root:Load pre-trained model from /source/main/train/output/saved_models/Seq2SeqAttnWithSrc/2019-06-01T16:23:45/470000.pt successfully
INFO:root:Model architecture: 
Seq2SeqBeamAttnWithSrc(
  (encoder): Encoder(
    (embedding): Embedding(25390, 256)
    (lstm): LSTM(256, 512, num_layers=3, dropout=0.3, bidirectional=True)
    (dropout): Dropout(p=0.3)
  )
  (flatten_hidden_lstm): FlattenHiddenLSTM()
  (core_decoder): AttnRawDecoderWithSrc(
    (dec_embedding): Embedding(50437, 256)
    (lstm): LSTM(512, 512, num_layers=3, dropout=0.3)
    (attention): Attention(
      (scoring): Linear(in_features=512, out_features=1024, bias=True)
      (softmax): Softmax()
    )
    (shrink_mapping): Linear(in_features=1536, out_features=512, bias=True)
    (output_mapping): Linear(in_features=512, out_features=50437, bias=True)
    (dropout): Dropout(p=0.3)
  )
  (infer_module): BeamSearchWithSrcInfer(
    (core_decoder): AttnRawDecoderWithSrc(
      (dec_embedding): Embedding(50437, 256)
     

In [5]:
def docs2input_tensors(preprocessed_docs, device):
    word_input = my_dataset.voc_src.docs2idx(preprocessed_docs, equal_length=MAX_LEN)
    word_input = np.array(word_input)
    word_input = torch.from_numpy(word_input)
    word_input = word_input.to(device)
    return word_input

def replace_unk_tok(pred, src):
    pred = [p if p != '__o__' else s for p, s in zip(pred.split(), src.split())]
    return ' '.join(pred)

def predict_batch(docs):
    docs = preprocessor.infer_preprocess(docs, max_length=MAX_LEN)
    input_tensors = docs2input_tensors(docs, device)
    predict_tensor = model(input_tensors)
    predict_numpy = predict_tensor.cpu().numpy()
    
    translated_docs = my_dataset.voc_tgt.idx2docs(predict_numpy)
    translated_docs = [' '.join(pred_doc.split()[:len(src_doc.split())]) 
                       for src_doc, pred_doc in zip(docs, translated_docs)]
    translated_docs = [replace_unk_tok(pred, src) for pred, src in zip(translated_docs, docs)]
    return translated_docs

def predict_docs(docs, batch_size):
    return list(chain(*[predict_batch(docs[i: i+batch_size]) for i in range(0, len(docs), batch_size)]))


In [6]:
MAX_LEN = 200
# import pdb; pdb.set_trace()
predict_docs(['toi muon dang ky the ATM', 'toi di hoc'], batch_size=10)

['tôi muốn đăng ký thẻ ATM', 'tôi đi học']

In [7]:
def get_metrics(df):
    logging.info('Total sentences: %s', df.shape[0])
    sen_acc = (df['tgt'] == df['pred']).sum()/df.shape[0]
    
    df = df[df['tgt'].map(lambda x: len(x.split())) == df['pred'].map(lambda x: len(x.split()))]
    logging.info('Total predicted sequences without changing len: %s', df.shape[0])
    tok_tgt = [tok for doc in df['tgt'] for tok in doc.split()]
    tok_pred = [tok for doc in df['pred'] for tok in doc.split()]
    sen_tok = (np.array(tok_tgt) == np.array(tok_pred)).sum()/len(tok_tgt)
    
    return sen_acc, sen_tok

# Predict

In [8]:
df = pd.read_csv('/source/main/data_for_train/output/my_test.csv')

In [9]:
start = time.time()
pred = predict_docs(list(df['src']), batch_size=128)
end = time.time()
df['pred'] = pred
logging.info('Duration: %.2f s' % (end-start))

INFO:root:Duration: 44.73 s


In [10]:
get_metrics(df)

INFO:root:Total sentences: 5000
INFO:root:Total predicted sequences without changing len: 5000


(0.3612, 0.9710605570107027)

- Baseline/2019-05-04T01:16:45:
  + 150k: 0.383, 0.9732856714953901
  + 180k: 0.3934, 0.9739834829348896
  + 120k (version on gpu): 0.3726, 0.9717781354609999
- SimpleButHuge/2019-05-04T01:40:37:
  + 80k: 0.3714, 0.9716497644886392
  + 150k: 0.4134, 0.9751091976155914
  
- Seq2SeqAttn:
  + (0.0222, 0.7263146191602002)
  
- Seq2SeqAttnWithSrc:
  + Step 320k: (0.3524, 0.9709428372426107)
  + Step 320k: (0.3296, 0.9689187112301388)
  + Step 470k: (0.3612, 0.9710605570107027)  - test: 0.97119

In [ ]:
tgt = [text_utils.process_line(doc) for doc in df['tgt']]
tgt = list(zip(*tgt))[1]
# tgt = list(chain(*tgt))
pred = [text_utils.process_line(doc) for doc in df['pred']]
pred = list(zip(*pred))[1]
# pred = list(chain(*pred))

In [ ]:
df['tgt_'] = tgt
df['pred_'] = pred

In [ ]:
df[df['tgt_'].map(len) != df['pred_'].map(len)].shape

In [ ]:
df_ = df[df['tgt_'].map(len) == df['pred_'].map(len)]

In [ ]:
tgt = list(chain(*df_['tgt_']))
pred = list(chain(*df_['pred_']))

In [ ]:
len(tgt)

In [ ]:
len(pred)

In [ ]:
(np.array(tgt) == np.array(pred)).sum()/len(tgt)

- SimpleButHuge/2019-05-04T01:40:37:
 + 470k: 0.9750297875460562

# Predict to summit

In [30]:
with open('/dataset/vietnamese_tone_prediction/test.txt', 'rt', encoding='utf-8') as i_f:
    lines = i_f.readlines()
lines = [l[:-1] for l in lines]
lines = [(l[:3], l[4:]) for l in lines]
df_submit = pd.DataFrame(lines)

In [31]:

df_submit['len'] = df_submit[1].map(lambda x: len(x.split()))

In [32]:
df_submit['len'].describe()

count    8240.000000
mean     55.138592  
std      12.340192  
min      2.000000   
25%      47.000000  
50%      53.000000  
75%      62.000000  
max      173.000000 
Name: len, dtype: float64

In [33]:
df_submit['pred'] = predict_docs(list(df_submit[1]), batch_size=128)

In [34]:
df_submit.head()

,0,1,len,pred
0,axS,"Trung Quoc da mo rong anh huong cua ho trong khu vuc thong qua cac buoc leo thang ep buoc cac nuoc lang gieng o Hoa Dong, Bien Dong, boi dap dao nhan tao va quan su hoa cac cau truc dia ly tren Bien Dong trai luat phap quoc te; Tim cach chia re Hoa Ky khoi cac dong minh chau A thong qua cac no luc ep buoc va leo lai kinh te.",75,"Trung Quốc đã mở rộng ảnh hưởng của họ trong khu vực thông qua các bước leo thang ép buộc các nước láng giềng ở Hoa Đông , Biển Đông , bởi đập đảo nhân tạo và quân sự hóa các cấu trúc địa lý trên Biển Đông trái luật pháp quốc tế ; Tìm cách chia rẽ Hoa Kỳ khỏi các đồng minh châu Á thông qua các nỗ lực ép buộc và lèo lái kinh tế ."
1,fqk,"De san xuat nuoc hoa gia cac thuong hieu noi tieng, doi tuong Tu Anh da mua cac hoa chat co nguon goc tu Trung Quoc ve de u lam tang do thom roi dung cac may moc dong goi, mua vo, nhan hieu cua cac thuong hieu noi tieng de dan len, sau do dua ra thi truong tieu thu.",61,"Để sản xuất nước hoa giả các thương hiệu nổi tiếng , đối tượng Tú Anh đã mua các hóa chất có nguồn gốc từ Trung Quốc về để ủ làm tăng độ thơm rồi dùng các máy móc đóng gói , mua vỏ , nhãn hiệu của các thương hiệu nổi tiếng để đan lên , sau đó đưa ra thị trường tiêu thụ ."
2,yFK,"Tuy nhien, tren thuc te, moi nguoi deu hieu rang, viec cuoc cai to noi cac lan nay cua ba May la mot canh bac nham xac dinh va ap dat quyen luc lanh dao cua ba doi voi nhung thanh vien noi cac, trong do co nhung nguoi da the hien bat dong chinh kien voi ba trong van de Brexit va mot so van de khac ve chinh tri, kinh te, xa hoi.",75,"Tuy nhiên , trên thực tế , mọi người đều hiểu rằng , việc cuộc cải tổ nội các lần này của bà May là một canh bạc nhằm xác định và áp đặt quyền lực lãnh đạo của ba đối với những thành viên nội các , trong đó có những người đã thể hiện bất đồng chính kiến với ba trong vấn đề Brexit và một số vấn đề khác về chính trị , kinh tế , xã hội ."
3,gdm,"Nguyen Xuan Hai, tinh phuc tap trong viec huy dong von cho dau tu ha tang khong chi dung o viec chon mot co che tai chinh hop ly, ma con o viec tan dung cac co che khac nhau tai cac thoi diem khac nhau cho moi du an.",50,"Nguyễn Xuân Hải , tính phức tạp trong việc huy động vốn cho đầu tư hạ tầng không chỉ dừng ở việc chọn một cơ chế tài chính hợp lý , mà còn ở việc tận dụng các cơ chế khác nhau tại các thời điểm khác nhau cho mọi dự án ."
4,Pmo,"Anh: Goal Bayern san hang noi dia de chia lua cho Lewandowski: HLV Jupp Heynckes thua nhan ban lanh dao Bayern Munich da bat dau xuc tien ke hoach chieu mo tien dao nguoi Duc dang choi cho Hoffenheim, Sandro Wagner.",41,"Ảnh : Goal Bayern săn hạng nội địa để chia lửa cho Lewandowski : HLV Jupp Heynckes thừa nhận ban lãnh đạo Bayern Munich đã bắt đầu xúc tiến kế hoạch chiêu mộ tiền đạo người Đức đang chơi cho Hoffenheim , Sandro Wagner ."


In [35]:
# df_submit.loc[967, 'pred'] = 'Nếu fan của thể loại nhạc không lời đậm chất sự thì thì chắc chắn ai cũng sẽ biết tới Two Steps from Hell công ty chuyển sang tác nhạc cho trailer phim , đã góp phần thành công cho nhiều bom tấn như : series Harry Potter , The Dark Knight , Rise of the Planet of the Apes , Trộn : Time , Wall-E . . . Cùng với đó , không thể không nhắc đến sự góp mặt của x - ca sĩ nổi tiếng người Mỹ .'
# df_submit.loc[526, 'pred'] = 'Tiếp tục triển khai cấp kè trên toàn thành phố , trong đó chú trọng hoạt động cấp lưu động cho nhân dân tại nhà , trường học và các khu dân cư ; phối hợp bưu điện trả x theo địa chỉ nhân dân yêu cầu , giảm thời gian đi lại của người dân .'

In [36]:
with open('/source/main/data_download/output/test_pred.txt', 'wt', encoding='utf-8') as o_f:
    o_f.writelines(['%s,%s\n' % (id_doc, doc) for id_doc, doc in zip(df_submit[0], df_submit['pred'])])

In [37]:
text_utils.decompose_predicted_test_file('/source/main/data_download/output/test_pred.txt')

In [38]:
df1 = pd.read_csv('/dataset/vietnamese_tone_prediction/test_word_per_line.txt')

In [39]:
df2 = pd.read_csv('/source/main/data_download/output/test_pred_no_tone.csv')

In [40]:
set(df1['id'])^set(df2['id'])

set()

In [ ]:
set(df2['id'])^set(df1['id'])

In [ ]:
doc_id='icy'

In [ ]:
' '.join(list(df2[df2['id'].map(lambda x: x[:3]==doc_id)]['no_tone']))

In [ ]:
' '.join(list(df1[df1['id'].map(lambda x: x[:3]==doc_id)]['no_tone']))

In [ ]:
df_submit[df_submit[0]==doc_id]

In [ ]:
!ls /source/main/data_download/output/

In [41]:
!wc -l /source/main/data_download/output/test_pred_simplified.csv

453447 /source/main/data_download/output/test_pred_simplified.csv


In [ ]:
predict_batch(['__o__'])